In [ ]:
# 1. UNINSTALL EVERYTHING (Clear the conflicts)
!pip uninstall -y trl peft transformers accelerate bitsandbytes

# 2. INSTALL LATEST STABLE VERSIONS (From PyPI, not Git)
# These versions are guaranteed to work together for Gemma 2
!pip install -q -U torch
!pip install -q -U bitsandbytes
!pip install -q -U transformers==4.47.0
!pip install -q -U peft==0.14.0
!pip install -q -U trl==0.13.0
!pip install -q -U accelerate

print("✅ Fresh Start Installed.")
print("⚠️ CRITICAL: You MUST restart the session now. (Runtime -> Restart Session)")

Found existing installation: trl 0.9.4
Uninstalling trl-0.9.4:
  Successfully uninstalled trl-0.9.4
Found existing installation: peft 0.18.1.dev0
Uninstalling peft-0.18.1.dev0:
  Successfully uninstalled peft-0.18.1.dev0
Found existing installation: transformers 4.46.0
Uninstalling transformers-4.46.0:
  Successfully uninstalled transformers-4.46.0
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: bitsandbytes 0.49.0
Uninstalling bitsandbytes-0.49.0:
  Successfully uninstalled bitsandbytes-0.49.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 123.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 21.8 MB/s eta 0:0

In [ ]:
import torch
import shutil
import os
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig

In [ ]:
# --- CONFIGURATION ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

DATASET_FILE = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/psychologist_dataset_ready.jsonl"
MODEL_ID = "MBZUAI-Paris/Atlas-Chat-9B"
NEW_MODEL_NAME = "Atlas-Psychologist-Darija-v1"
DRIVE_BACKUP_PATH = f"/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/{NEW_MODEL_NAME}"

# 1. Load Dataset
full_dataset = load_dataset("json", data_files=DATASET_FILE, split="train")
dataset_dict = full_dataset.train_test_split(test_size=0.1)
train_dataset = dataset_dict["train"]
eval_dataset = dataset_dict["test"]

Mounted at /content/drive


In [ ]:
# 2. Formatting Function
def formatting_prompts_func(examples):
    output_texts = []
    for instruction, input_text, response in zip(examples['instruction'], examples['input'], examples['output']):
        text = f"<start_of_turn>user\n{instruction}\n\nPatient says: \"{input_text}\"<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn>"
        output_texts.append(text)
    return output_texts

In [ ]:
# 3. Load Model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.12/logging/__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/logging/__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/logging/__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/usr/l

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 4. LoRA Config
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "up_proj", "down_proj"]
)

In [ ]:
# 5. Training Arguments (Classic Style)
training_args = SFTConfig(
    output_dir="/content/drive/MyDrive/Colab Notebooks/NLP_PROJECT/results",
    max_seq_length=512,          # <--- Defined here for TRL 0.13.0
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    report_to="none",
    dataset_text_field="text",
)

In [ ]:
print("🚀 Starting Training...")
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    # CRITICAL CHANGE: Use 'processing_class' instead of 'tokenizer' for modern TRL
    processing_class=tokenizer,
    args=training_args,
)

trainer.train()

🚀 Starting Training...


Map:   0%|          | 0/5040 [00:00<?, ? examples/s]

Map:   0%|          | 0/561 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class in

KeyboardInterrupt: 

In [ ]:
# 7. Save & Backup
print("💾 Saving...")
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)
print(f"☁️ Backing up to Google Drive: {DRIVE_BACKUP_PATH}")
shutil.copytree(NEW_MODEL_NAME, DRIVE_BACKUP_PATH, dirs_exist_ok=True)
print("✅ Done!")

💾 Saving...
☁️ Backing up to Google Drive: /content/drive/MyDrive/Colab Notebooks/NLP_PROJECTAtlas-Psychologist-Darija-v1
✅ Done!


# Testing the psy

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# --- CONFIGURATION ---
BASE_MODEL_ID = "MBZUAI-Paris/Atlas-Chat-9B"
# This path must match where you saved your adapter
ADAPTER_PATH = "/content/drive/MyDrive/Colab Notebooks/NLP_PROJECTAtlas-Psychologist-Darija-v1"

# 1. Load the Base Brain (Atlas)
print("⏳ Loading the Base Model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=True,
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

# 2. Load Your "Psychologist" Personality
print("🧠 Loading your Fine-Tuned Adapter...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

# 3. Define the Chat Function
def ask_psychologist(user_input):
    # We must use the exact same format used during training
    prompt = f"<start_of_turn>user\nAct as a Moroccan psychologist and help this user.\n\nPatient says: \"{user_input}\"<end_of_turn>\n<start_of_turn>model\n"

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,       # Controls creativity (0.7 is balanced)
        top_p=0.9,
        repetition_penalty=1.1, # Prevents repeating phrases
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode and clean up the response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the psychologist's part
    response_only = full_response.split("model\n")[-1].strip()
    return response_only

# 4. Chat Loop
print("\n✅ The Doctor is IN. (Type 'exit' to quit)")
while True:
    user_text = input("\n👤 You (Darija): ")
    if user_text.lower() in ["exit", "quit", "salina"]:
        break

    response = ask_psychologist(user_text)
    print(f"🧠 Psychologist: {response}")

⏳ Loading the Base Model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

🧠 Loading your Fine-Tuned Adapter...

✅ The Doctor is IN. (Type 'exit' to quit)

👤 You (Darija): Danya dayra biya w ma3ndi m3amen nhder


The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


🧠 Psychologist: صافي ا صاحبي، راني فاهم شعورك. حاول تاخد نفس عميق وركز على تنفسك شوية باش تهدا راسك. كلنا كنمرو بهاد المشاعر من وقت لآخر، ولكن راه مهم نتعاملو معاهوم بطريقة صحية.

👤 You (Darija): Bach katnse7ni ?
🧠 Psychologist: هادشي مفهوم، الحب يقدر يكون قوي بزاف. ولكن راه مهم تهتم براسك وتلقى التوازن فحياتك. نتا تستاهل السعادة والرضا، وكاينين طرق صحية باش توصل ليهم. أنا هنا معاك وغادي نعاونك تلقى الحلول اللي كتناسبك.

👤 You (Darija): Ana fer7an 
🧠 Psychologist: مبروك عليك هاد الفرحة لي كتحس بيها! واش كاين شي حاجة خاصة خلاتك تحس بهاد السعادة الكبيرة؟


KeyboardInterrupt: Interrupted by user